### Гипотезы
1. Убрать ненужные слова по типу: расскажи, докажи, из, ...

    Есть два пути: выбрать опорные слова из определений и оставить только их или собрать служебные и второстепенные слова и выкинуть служебные
2. На каком языке лучше (хотим русский)
3. Дообучить русскую нейронку (недавно вышла Т-pro)

## качаю нужные библиотеки

In [54]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.7 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=6e58a6642933c1b236b8f53fac622f161774b4493cf4dc34626bc2d2259dee76
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [71]:
!pip install pdfminer.six

In [56]:
!pip install 'pdfminer.six[image]'

In [87]:
!pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


## всякие импорты

In [93]:
import re # регулярки
import nltk # для стоп-слов
from nltk.tokenize import sent_tokenize # для токенизации
from nltk.tokenize import RegexpTokenizer # разбиваем на слова
from nltk.corpus import stopwords # убираем стоп слова
import pymorphy2 # для лемантизации
from collections import Counter # подсчёт частых
from gensim.models.phrases import Phrases, Phraser # дли биграмм
from scipy import spatial
import numpy as np
import tensorflow as tf

from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from pdfminer.high_level import extract_text # pdf в текст

from google.colab import drive # подключение к диску

In [58]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [59]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [60]:
stopwords_ru = stopwords.words('russian')
tokenizer = RegexpTokenizer('\w+')
morph = pymorphy2.MorphAnalyzer()

## начинаем работу с текстами

### книга

In [61]:
# лекции за 2023 год + материалы итмо + книга хопкрофта на русском
all_texts = []

for i in range(1, 69):
    t = extract_text(f"/content/drive/MyDrive/texts/{i}.pdf")
    # глазами зацепилась за не оч хорошие переносы, поэтому фикс
    t = t.replace('-\n', '').replace('\n\n', '\n')
    all_texts.append(t)

## код

### какие-то мои функции

In [62]:
# подсчёт метрики k
def my_metrics(user, db, clean_dict=None, trash_dict=None):
    user = make_clean(user)
    db = make_clean(db)

    user = get_words_only(user)
    db = get_words_only(db)

    if clean_dict is not None:
        user = [w for w in user if w in my_dict]
        db = [w for w in db if w in my_dict]

    if trash_dict is not None:
        user = [w for w in user if w not in trash_dict]
        db = [w for w in db if w not in trash_dict]

    print(user)
    print(db)

    in_both = list(set(user) & set(db))

    return len(in_both) / (len(db) + epsilon)

In [63]:
# берём только слова из предложений
def get_words_only(sentences_tokenize):
    words = [item for sent in sentences_tokenize for item in sent]

    return words

In [64]:
def make_clean(text):
    # чистим текст
    text = text.lower()
    text = re.sub('\n|\t|\r', ' ', text)
    # преобразуем в предложения
    sentences = sent_tokenize(text)
    # бъём предложения на слова
    sentences_tokenize = [tokenizer.tokenize(item) for item in sentences]
    # убираем стоп слова
    sentences_tokenize = [[item for item in sent if (item not in stopwords_ru and re.match('^[а-я]*$', item))] for sent in sentences_tokenize]
    # лемантизируем слова
    sentences_tokenize = [[morph.normal_forms(item)[0] for item in sent] for sent in sentences_tokenize]

    return sentences_tokenize

In [65]:
# словарь популярных слов
def get_dict_words(sentences_tokenize):
    words = [item for sent in  sentences_tokenize for item in sent]
    word_dict = Counter(words)
    common = word_dict.most_common()

    return common

In [66]:
# полезные слова = часто встречаем их
def get_useful(common):
    my_dict = {}

    for i, j in common:
        if i != 'ε' and len(i) > 2 and re.match('^[а-я]*$', i):
            my_dict[i] = j

    return my_dict

### смотрим на текст

In [67]:
epsilon = 0.001

all_sentences = []
all_words = []

my_dict = {}

# мусорные конструкции
trash_dict = ['че', 'доказать', 'рассказать', 'ваш', 'привести', 'сделать', 'описать', 'дать', 'что']

In [68]:
for text in all_texts:
    t = make_clean(text)
    if len(t) > 0:
        all_sentences = all_sentences + t

    # частые слова
    common = get_dict_words(t)
    all_words += [i for (i, j) in common]
    # «полезные» слова
    useful_dict = get_useful(common)

    new_dict = dict(list(my_dict.items()) + list(useful_dict.items()))
    my_dict = new_dict.copy()

### предобработка перед doc2vec

In [82]:
vocab_index = [TaggedDocument(text,[k]) for k, text in enumerate(all_sentences)]

model = Doc2Vec(vector_size=300, window=1, min_count=3, workers=6, epochs=20)

model.build_vocab(vocab_index)
model.train(vocab_index, total_examples=model.corpus_count, epochs=20)

In [98]:
s1 = model.infer_vector('дка'.split())
s2 = model.infer_vector('регулярные языки и их свойства'.split())

s1 = tf.math.l2_normalize(s1).numpy()
s2 = tf.math.l2_normalize(s2).numpy()

cos_distance = spatial.distance.cosine(s1, s2)

cos_distance

0.9227588153707628

In [99]:
similar_vec = model.docvecs.most_similar([s2], topn=10)

similar_sent = [" ".join(vocab_index[top[0]].words) for top in similar_vec]

<ipython-input-99-29078a78222d>:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_vec = model.docvecs.most_similar([s2], topn=10)


### предобработка перед w2v

In [ ]:
bigram = Phrases(all_sentences)
bigram_transformer = Phraser(bigram)

# генератор текстов с биграммами
def bigram_generator():
    for text in all_sentences:
        yield bigram_transformer[[word for word in text]]

In [ ]:
trigram = Phrases(bigram_generator())
trigram_transformer = Phraser(trigram)

def trigram_generator():
    for text in all_sentences:
        yield trigram_transformer[bigram_transformer[[word for word in text]]]

In [ ]:
ok = [i for i in trigram_generator()]

In [ ]:
model = Word2Vec(vector_size=300, window=10, min_count=3, workers=4)
model.build_vocab(ok)

In [ ]:
model.corpus_count

21753

In [ ]:
model.train(ok, total_examples=model.corpus_count, epochs=500)

(56711352, 67266500)

In [ ]:
model.wv.most_similar('дка')

[('нка', 0.47808364033699036),
 ('автомат', 0.4368232488632202),
 ('состояние', 0.3628830313682556),
 ('процесс_допуск', 0.2999037206172943),
 ('нда', 0.2974991500377655),
 ('множество', 0.2651441693305969),
 ('так_быть', 0.2553767263889313),
 ('эквивалентный_дка', 0.24655848741531372),
 ('заключительный_состояние', 0.24137888848781586),
 ('начальный_состояние', 0.2403261363506317)]

In [ ]:
# сохраню потом
model.save('wv_model')

In [ ]:
trigram_transformer.save('wv_trigramm')

# подвал

In [ ]:
# чистим текст из пдфки
test = make_clean(all_texts[0])

In [ ]:
# словарь с частыми словами
common = get_dict_words(test)

In [ ]:
# словарь с полезными терминами
my_dict = get_useful(common)

In [36]:
s1 = 'че такое эти ваши производные антимирова'

s2 = 'Как можно вычислить частичные производные Антимирова?'

In [ ]:
my_metrics(s1, s2, trash_dict=trash_dict)

['такой', 'производный', 'антимиров']
['вычислить', 'частичный', 'производный', 'антимиров']


0.4998750312421894

и тут для меня дошло, что, если расписывать вопросы в БЗ, то деление на длину слов вопроса из БЗ (коих будет много) не будет явно и точно отражать близость, но относительно такой штуки можно смотреть динамику

In [72]:
!pip install FlagEmbedding

In [73]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [74]:
from sentence_transformers import SentenceTransformer

mmm = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [75]:
sentences_1 = ['че такое эти ваши производные антимирова',
               "великая и всемогущая модель, расскажи мне про регулярные языки",
               "правда, что любой язык детерминирован?"]
sentences_2 = ['Как можно вычислить частичные производные Антимирова?',
               "расскажи деревья разбора",
               "Что представляет собой язык, распознаваемый недетерминированным конечным автоматом (НКА)?"]

In [76]:
embeddings_1 = model.encode(sentences_1)['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [77]:
em_1 = mmm.encode(sentences_1)
em_2 = mmm.encode(sentences_2)

In [78]:
for i in range(len(embeddings_1)):
    for j in range(len(embeddings_2)):
        e_1 = embeddings_1[i]
        e_2 = embeddings_2[j]
        st_1 = em_1[i]
        st_2 = em_2[j]
        s_1 = sentences_1[i]
        s_2 = sentences_2[j]
        print(f'{s_1} AND {s_2}: ', e_1 @ e_2, st_1 @ st_2)

че такое эти ваши производные антимирова AND Как можно вычислить частичные производные Антимирова?:  0.6444841 0.85528797
че такое эти ваши производные антимирова AND расскажи деревья разбора:  0.32307824 0.5920198
че такое эти ваши производные антимирова AND Что представляет собой язык, распознаваемый недетерминированным конечным автоматом (НКА)?:  0.26799577 0.6695769
великая и всемогущая модель, расскажи мне про регулярные языки AND Как можно вычислить частичные производные Антимирова?:  0.3195983 0.6621109
великая и всемогущая модель, расскажи мне про регулярные языки AND расскажи деревья разбора:  0.46826863 0.6026131
великая и всемогущая модель, расскажи мне про регулярные языки AND Что представляет собой язык, распознаваемый недетерминированным конечным автоматом (НКА)?:  0.49805534 0.656705
правда, что любой язык детерминирован? AND Как можно вычислить частичные производные Антимирова?:  0.30123478 0.6517712
правда, что любой язык детерминирован? AND расскажи деревья разбора:  